In [ ]:
!pip install split-folders tqdm

In [ ]:
import os
import math
import cv2
import shutil
import numpy as np
import pandas as pd
import skimage.io
from PIL import Image
from tqdm.notebook import tqdm
import zipfile
import tifffile
import matplotlib.pyplot as plt

import splitfolders

In [ ]:
# making output directories

os.mkdir('./Images')
os.mkdir('./Masks')

In [ ]:
splitfolders.ratio("../input/copy-of-iafoss-dataset-256x256/train", output="./Images", seed=1337, ratio=(.8, .1, .1), group_prefix=None)

In [ ]:
splitfolders.ratio("../input/copy-of-iafoss-dataset-256x256/masks", output="./Masks", seed=1337, ratio=(.8, .1, .1), group_prefix=None)

In [ ]:
os.listdir('./Images/')

In [ ]:
import os
list = os.listdir('./Images/train/Images_01') # dir is your directory path
number_files1 = len(list)
print(number_files1)

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
%env SM_FRAMEWORK=tf.keras

import keras
import tensorflow as tf
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from keras.models import model_from_json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler

from keras.layers import Input, Conv2D, Reshape
from keras.models import Model
from keras import backend as K

In [ ]:
#Preprocessing 'train_images' for segmentation model

BACKBONE = 'resnet34'
preprocess_input = get_preprocessing(BACKBONE)

In [ ]:
data_gen_args = dict(
#     featurewise_center=True,
#                      featurewise_std_normalization=True,
                     rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)

image_datagen = preprocess_input(ImageDataGenerator(**data_gen_args))
mask_datagen = preprocess_input(ImageDataGenerator(**data_gen_args))

In [ ]:
# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1

In [ ]:
train_image_generator = image_datagen.flow_from_directory(
    './Images/train',
    class_mode=None,
    batch_size=32,
    seed=seed)
train_mask_generator = mask_datagen.flow_from_directory(
    './Masks/train',
    class_mode=None,
    batch_size=32,
    seed=seed)

# combine generators into one which yields image and masks
train_generator = zip(train_image_generator, train_mask_generator)

In [ ]:
validation_image_generator = image_datagen.flow_from_directory(
    './Images/val',
    class_mode=None,
    batch_size=32,
    seed=seed)
validation_mask_generator = mask_datagen.flow_from_directory(
    './Masks/val',
    class_mode=None,
    batch_size=32,
    seed=seed)

# combine generators into one which yields image and masks
validation_generator = zip(validation_image_generator, validation_mask_generator)

In [ ]:
# Model Building using U-net

from keras.layers import Reshape

# create the base pre-trained model
base_model = Unet(backbone_name='resnet34', encoder_weights='imagenet')

# add a layer with defined input shape
input_base_model = Input(shape=(256, 256, 3))

# add a convolution layer with input data
l1 = Conv2D(3, (1, 1))(input_base_model)

# defining output layer shape
out = base_model(l1)

model = Model(input_base_model, out, name=base_model.name)

In [ ]:
model.summary()

In [ ]:
# callbacks

lr_schedule = keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 10))

lr_check = keras.callbacks.ReduceLROnPlateau(patience = 4)

# initial_learning_rate = 0.01
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=100000,
#     decay_rate=0.96,
#     staircase=True)


early_stopping = keras.callbacks.EarlyStopping(patience=8 , verbose = 1)

model_checkpoint = keras.callbacks.ModelCheckpoint(
                   '/kaggle/working/best_cnn.h5', 
                   save_best_only=True)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001, amsgrad=False)

In [ ]:
# Defining metrics and loss

def dice_coefficient(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon())

# def loss(y_true, y_pred):
#     return binary_crossentropy(y_true, y_pred) - tf.math.log(dice_coefficient(y_true, y_pred) + tf.keras.backend.epsilon())

In [ ]:
# Model Compiling

model.compile(optimizer = optimizer, loss=dice_loss, metrics=[dice_coefficient])

In [ ]:
# Data augmentation

datagen = ImageDataGenerator(rotation_range=8,
                             zoom_range=[0.95, 1.05],
                             height_shift_range=0.10,
                             shear_range=0.15)

In [ ]:
# Fit model in order to determine best learning rate 

history = model.fit(train_generator,steps_per_epoch = 32,
                                  epochs=30, 
                                  validation_data=validation_generator,
                                  validation_steps = 8,
                                  callbacks=[lr_check, early_stopping, model_checkpoint]
                                 )

In [ ]:
test_image_generator = image_datagen.flow_from_directory(
    './Images/test',
    class_mode=None,
    batch_size = 32,
    seed=seed)
test_mask_generator = mask_datagen.flow_from_directory(
    './Masks/test',
    class_mode=None,
    batch_size = 32,
    seed=seed)

test_generator = zip (test_image_generator , test_mask_generator)

In [ ]:
model.evaluate(test_generator)

# not working

In [ ]:
model.metrics_names

In [ ]:
dir(history)

In [ ]:
history.history

In [ ]:
dice_coefficient = history.history['dice_coefficient']
val_dice_coefficient = history.history['val_dice_coefficient']

train_loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:

plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.plot(dice_coefficient, label='Dice coefficient')
plt.plot(val_dice_coefficient, label='Validation Dice coefficient')
plt.legend()
plt.xlabel('Epochs')
plt.title('Epochs vs. Training and Validation Dice coefficient')
    
plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.title('Epochs vs. Training and Validation Loss')

plt.show()

In [ ]:
# Plot Learning Rate vs. Loss

plt.plot(history.epoch, history.history['loss'])
# plt.axis([1e-4, 1e-1, 0, 4])
plt.xlabel('Epochs')
plt.ylabel('Training Loss')
plt.show()